NumPyはPythonにおける高速数値計算用のライブラリである。型付き多次元配列オブジェクト (numpy.ndarray)や、数学関数などが使える。SciPyやmatplotlibなど、NumPyに基づくライブラリは多い

---

# 資料集

* [朱鷺の杜Wiki](http://ibisforest.org/index.php?python%2Fnumpy)
    - これ見ればよくね
* [NumPy 利用ノート](https://showa-yojyo.github.io/note/python-numpy.html)

# 高速化
NumPyの計算をより高速にするにはいくつかの方法がある
* コードの書き方を工夫する
* BLAS, LAPACKを高速なものに取り換える
    - BLAS (Basic Linear Algebra Subprograms)は行列の積といった基本的な線形代数計算を高速で行うライブラリであり、主にFortranやCで実装されている（[BLASとは](http://azalea.s35.xrea.com/blas/whatIsBlas.html)）。様々な実装があり、有料のIntel MKL (Math Kernel Library)が最強だが、無料ではGotoBLAS系列(GotoBLAS→GotoBLAS2→OpenBLAS)またはATLASが良く使われる。昔のものは昔のCPUにチューンされているので避けよう。2014年時点ではATLASよりOpenBLASの方が速いっぽい（[OpenBLAS と ATLAS の性能を R 上で比較する](http://www.hnagata.net/archives/198)）
    - LAPACK (Linear Algebra PACKage)はBLASを呼び出して固有値・特異値問題といったより高度な線形代数計算を高速で行うライブラリ。LAPACK付属のBLASは遅いので、自分で用意した速いBLASに取り換えよう
    - これらはPython以前からある歴史の長いライブラリである。C/C++ではGSL(GNU Scientific Library)という科学計算用のライブラリを通じて使うことが多い
* Cythonを使う
    - [NumPy を扱う](http://omake.accense.com/static/doc-ja/cython/src/tutorial/numpy.html)
    - [NumPyとCythonを組み合わせると爆速！](http://kesin.hatenablog.com/entry/20120314/1331689014)
    - [[Python][Cython]メモ: numpyアクセス高速化](http://nekopuni.holy.jp/2014/10/pythoncython%E3%83%A1%E3%83%A2-numpy%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9%E9%AB%98%E9%80%9F%E5%8C%96/)

### Open BLASの導入
* [numpy・scipyでマルチスレッドBLAS・LAPACKを使う](http://qiita.com/_akisato/items/d5de60e38b5a69387bde) 
* [Numpy/Scipy with OpenBLAS for Ubuntu 12.04](http://osdf.github.io/blog/numpyscipy-with-openblas-for-ubuntu-1204.html)
* [Building NumPy and SciPy with GotoBLAS2](https://lucidfrontier45.wordpress.com/tag/blas/)
* [numpy, scipyちょっと遅い気がするって時はblas, lapack, atlasが入っているか見直しましょう](http://qiita.com/r9y9/items/b6b2a0ee00781f1bb480)